In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

In [2]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	#trainY = to_categorical(trainY)
	#testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [3]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [4]:
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)

In [5]:
clf1 = tf.keras.models.load_model("cifar-vgg-baseline-holdout-100epochs_all_transfer.npy")

In [6]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [7]:
clf2 = tf.keras.models.load_model("cifar-mobileNet-baseline-holdout-100epochs_all_transfer.npy")

In [8]:
clf3 = tf.keras.models.load_model("cifar-resnet50-baseline-holdout-400epochs_dropout_batch_augmentation.npy")

In [12]:
pr = clf3.predict(testX)

313/313 [==============================] - 9s 25ms/step


In [10]:
len(pr)

10000

In [20]:
class emsembleModel:
    def __init__(self, classifiers, numClasses):
        self.classifiers = classifiers
        self.numClasses = numClasses
        
    def predict(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            counting = [0 for _ in range(self.numClasses)]
            current_best_pred = preds[0][i]
            for j in range(M):
                pred = preds[j][i]
                predictedClass = np.argmax(pred)
                counting[predictedClass]+=1
                if counting[predictedClass] == max(counting):
                    current_best_pred = pred
            resultPred.append(current_best_pred)
        
        return resultPred
    
    def evaluate(self, dataX, dataY):
        pred = self.predict(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = dataY[i]
            predictedClass = np.argmax(pred[i])
            if expectedClass == predictedClass:
                ok += 1
        return ok/len(dataY)

In [19]:
#eclf = VotingClassifier(estimators=[('vgg', clf1), ('mobileNet', clf2), ('resnet50', clf3)], voting='hard')

In [21]:
# for clf, label in zip([clf1, clf2, clf3, eclf], ['VGG', 'MobileNet', 'Resnet50', 'Ensemble']):
#     scores = cross_val_score(clf, testX, testY, scoring='accuracy', cv=5)
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000001C46CA5DF40>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [14]:
classifiers = [clf1, clf2, clf3]

In [21]:
emsemble = emsembleModel(classifiers, 10)

In [22]:
acc = emsemble.evaluate(testX, testY)

313/313 [==============================] - 7s 24ms/step


In [24]:
print('Accuracy Emsemble: %.3f' % (acc * 100.0))

Accuracy: 85.550


In [ ]:
ok = 0
for i in range(len(testX)):
    img = testX[i]
    sample = img.reshape((1,32,32,3))
    counting = [0 for _ in range(11)]
    for clf in classifiers:
        pred = clf.predict(sample, verbose=0)
        predictedClass = np.argmax(pred)
        counting[predictedClass]+=1
    
    best = counting.index(max(counting))
    if best == testY[i]:
        ok+=1

In [ ]:
acc = ok/len(testX)

In [ ]:
print('Accuracy Emsemble: %.3f' % (acc * 100.0))